In [76]:
# Importing Pandas and NumPy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [77]:
telecom = pd.read_csv("telecom_churn_data.csv", encoding = "ISO-8859-1", header= 0)
telecom.head(10)

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.40,0.00,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.00,0.00,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.00,0.00,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.00,0.00,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.00,0.00,0.00,0.00
5,7000286308,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,50.258,...,0,NaN,NaN,NaN,NaN,1471,0.00,0.00,0.00,0.00
6,7001051193,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,429.023,...,0,NaN,NaN,NaN,NaN,1673,0.00,0.00,0.00,0.00
7,7000701601,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,1069.180,...,0,NaN,NaN,NaN,NaN,802,57.74,19.38,18.74,0.00
8,7001524846,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,378.721,...,0,NaN,1.0,1.0,NaN,315,21.03,910.65,122.16,0.00
9,7001864400,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,119.518,...,0,NaN,1.0,NaN,NaN,902,0.00,0.00,0.00,0.00


# Data Preparation

In [78]:
telecom.shape

(99999, 226)

In [79]:
telecom.columns

Index(['mobile_number', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou',
       'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7',
       'last_date_of_month_8', 'last_date_of_month_9', 'arpu_6',
       ...
       'sachet_3g_9', 'fb_user_6', 'fb_user_7', 'fb_user_8', 'fb_user_9',
       'aon', 'aug_vbc_3g', 'jul_vbc_3g', 'jun_vbc_3g', 'sep_vbc_3g'],
      dtype='object', length=226)

In [80]:
telecom.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


In [81]:
#Remove all columns with only 0 and NaNs
telecom = telecom.loc[:,telecom.any()]

In [82]:
telecom.shape

(99999, 215)

In [83]:
#Removing few columns due to -:
#1. Circle_id not relevant since all are rows have same value
#2. Objective is to find churn based on usage, so revenue based columns not needed
#3. Stored last date of month in separate variables

cols_to_remove = ["circle_id",
"last_date_of_month_6",
"last_date_of_month_7",
"last_date_of_month_8",
"last_date_of_month_9",
"arpu_6",
"arpu_7",
"arpu_8",
"arpu_9",
"arpu_3g_6",
"arpu_3g_7",
"arpu_3g_8",
"arpu_3g_9",
"arpu_2g_6",
"arpu_2g_7",
"arpu_2g_8",
"arpu_2g_9"]

In [84]:
telecom = telecom.drop(cols_to_remove, axis = 1)

In [85]:
telecom.shape

(99999, 198)

###### Filtering out the High Value customers based on their recharge amount

In [86]:
#Adding columns for average recharge amount in months 6 and 7
telecom["avg_rech_amt_6_7"] = (telecom['total_rech_amt_6'] + telecom['total_rech_amt_7'])/2

In [87]:
# Calculating 70th percentile of the average value
final_avg_value_6_7 = telecom["avg_rech_amt_6_7"].quantile(0.7)
final_avg_value_6_7

368.5

In [88]:
telecom = telecom.loc[(telecom["avg_rech_amt_6_7"] > final_avg_value_6_7),:]

In [89]:
telecom.shape

(29979, 199)

###### Finding the churned customers based on data of 4th month

In [90]:
telecom["churn"] = (telecom["total_ic_mou_9"]==0) & (telecom["total_og_mou_9"]==0) & (telecom["vol_3g_mb_9"]==0) & (telecom["vol_2g_mb_9"]==0)

In [91]:
telecom.shape

(29979, 200)

In [92]:
telecom.loc[(telecom["churn"] == 1),:]

,mobile_number,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,offnet_mou_7,offnet_mou_8,offnet_mou_9,roam_ic_mou_6,...,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_rech_amt_6_7,churn
7,7000701601,57.84,54.68,52.29,NaN,453.43,567.16,325.91,NaN,16.23,...,NaN,NaN,NaN,802,57.74,19.38,18.74,0.0,1185.0,True
97,7000589828,433.59,415.66,221.06,NaN,74.54,43.66,31.86,NaN,0.00,...,NaN,NaN,NaN,502,0.00,0.00,0.00,0.0,380.0,True
111,7001300706,55.19,3.26,NaN,NaN,45.51,12.34,NaN,NaN,0.00,...,NaN,NaN,NaN,332,0.00,0.00,0.00,0.0,441.0,True
143,7000106299,1325.91,28.61,NaN,NaN,13.91,1.89,NaN,NaN,0.00,...,NaN,NaN,NaN,264,0.00,0.00,0.00,0.0,418.0,True
188,7000340381,4.38,0.98,NaN,NaN,105.16,39.39,NaN,NaN,0.00,...,NaN,NaN,NaN,244,0.00,831.48,1223.04,0.0,492.0,True
199,7002311591,288.56,376.66,111.61,7.13,186.59,1326.06,771.14,12.98,52.96,...,NaN,NaN,NaN,1145,0.00,0.00,0.00,0.0,633.0,True
320,7000959346,120.19,236.14,1.71,NaN,2082.18,2532.03,408.54,NaN,0.00,...,NaN,NaN,NaN,896,0.00,0.00,0.00,0.0,1828.5,True
358,7002255278,68.34,38.93,NaN,NaN,550.18,209.68,NaN,NaN,19.39,...,1.0,NaN,NaN,2597,1.22,17.89,383.36,0.0,750.0,True
478,7000104470,1241.99,1026.66,0.00,NaN,112.91,115.13,0.00,NaN,0.00,...,NaN,NaN,NaN,356,0.00,0.00,0.00,0.0,646.0,True
490,7000369789,189.51,NaN,NaN,NaN,920.18,NaN,NaN,NaN,0.00,...,NaN,NaN,NaN,341,0.00,0.00,0.00,0.0,393.0,True


In [93]:
# Removing the columns related to the 4th month
telecom = telecom.loc[:,~telecom.columns.str.endswith('_9')]

In [94]:
telecom["churn"] = telecom["churn"].astype(int)

In [95]:
telecom.head(10)

,mobile_number,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,...,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,avg_rech_amt_6_7,churn
7,7000701601,57.84,54.68,52.29,453.43,567.16,325.91,16.23,33.49,31.64,...,NaN,NaN,NaN,802,57.74,19.38,18.74,0.0,1185.0,1
8,7001524846,413.69,351.03,35.08,94.66,80.63,136.48,0.00,0.00,0.00,...,NaN,1.0,1.0,315,21.03,910.65,122.16,0.0,519.0,0
13,7002191713,501.76,108.39,534.24,413.31,119.28,482.46,23.53,144.24,72.11,...,NaN,NaN,1.0,2607,0.00,0.00,0.00,0.0,380.0,0
16,7000875565,50.51,74.01,70.61,296.29,229.74,162.76,0.00,2.83,0.00,...,NaN,NaN,NaN,511,0.00,2.45,21.89,0.0,459.0,0
17,7000187447,1185.91,9.28,7.79,61.64,0.00,5.54,0.00,4.76,4.81,...,NaN,NaN,NaN,667,0.00,0.00,0.00,0.0,408.0,0
21,7002124215,102.41,132.11,85.14,757.93,896.68,983.39,0.00,0.00,0.00,...,NaN,NaN,NaN,720,0.00,0.00,0.00,0.0,640.0,0
24,7001125315,124.19,55.19,141.11,450.33,370.48,283.36,9.30,0.00,13.83,...,NaN,NaN,NaN,3072,0.00,0.00,0.00,0.0,385.0,0
33,7000149764,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,1.0,1.0,1.0,1328,358.10,2706.83,1714.67,0.0,1932.0,0
38,7000815202,248.99,619.96,666.38,88.86,50.58,97.81,0.00,0.00,0.00,...,NaN,NaN,NaN,1155,0.00,0.00,0.00,0.0,528.0,0
41,7000721289,86.39,118.88,80.44,232.36,280.78,136.69,0.00,0.00,0.00,...,NaN,NaN,NaN,2010,0.00,0.00,0.00,0.0,503.0,0


###### Checking for missing values

In [96]:
#Checking for NaN values in the dataset
telecom.isnull().sum().sort_values(ascending=False)

date_of_last_rech_data_6    18592
count_rech_3g_6             18592
count_rech_2g_6             18592
av_rech_amt_data_6          18592
night_pck_user_6            18592
total_rech_data_6           18592
fb_user_6                   18592
max_rech_data_6             18592
count_rech_2g_7             18327
max_rech_data_7             18327
night_pck_user_7            18327
total_rech_data_7           18327
date_of_last_rech_data_7    18327
fb_user_7                   18327
av_rech_amt_data_7          18327
count_rech_3g_7             18327
night_pck_user_8            18238
av_rech_amt_data_8          18238
date_of_last_rech_data_8    18238
count_rech_3g_8             18238
count_rech_2g_8             18238
max_rech_data_8             18238
total_rech_data_8           18238
fb_user_8                   18238
loc_ic_t2t_mou_8              938
loc_og_mou_8                  938
std_og_t2t_mou_8              938
std_og_t2m_mou_8              938
loc_ic_t2m_mou_8              938
ic_others_8   

In [97]:
telecom.columns[telecom.isnull().sum()/len(telecom.index) > 0.3]

Index(['date_of_last_rech_data_6', 'date_of_last_rech_data_7',
       'date_of_last_rech_data_8', 'total_rech_data_6', 'total_rech_data_7',
       'total_rech_data_8', 'max_rech_data_6', 'max_rech_data_7',
       'max_rech_data_8', 'count_rech_2g_6', 'count_rech_2g_7',
       'count_rech_2g_8', 'count_rech_3g_6', 'count_rech_3g_7',
       'count_rech_3g_8', 'av_rech_amt_data_6', 'av_rech_amt_data_7',
       'av_rech_amt_data_8', 'night_pck_user_6', 'night_pck_user_7',
       'night_pck_user_8', 'fb_user_6', 'fb_user_7', 'fb_user_8'],
      dtype='object')

In [98]:
# Few columns are found to be important for our case study 
# Out of the cols where more than 30% data missing, it shows that almost 2/3 of customers dont use those services.
# The customers with data services are important and thus these cols cannot be deleted.
# Few cols with more than 60% missing data are irrevalent are in our study and are dropped.
del_cols = ["date_of_last_rech_data_6","date_of_last_rech_data_7","date_of_last_rech_data_8","max_rech_data_6","max_rech_data_7",
           "max_rech_data_8"]

In [99]:
# Drop columns with Null values greater than 30%
telecom = telecom.drop(del_cols, axis=1)

In [100]:
telecom.shape

(29979, 146)

In [101]:
newnulldf = telecom.columns[telecom.isnull().sum()/len(telecom.index) > 0.0]

###### Treating the missing values with zero 
After checking the dataset and according to the telecom domain, it can be concluded that the missing values are due to various factors -:
1> The customer may have joined at the 7th or 8th month, so no data available for other months.
2> Huge number of columns have more than 75% rows as zero.
3> Huge number of people dont use data services.

In [102]:
#Replace Nan values with zeros
dates = ["date_of_last_rech_6", "date_of_last_rech_7", "date_of_last_rech_8"]
for col_name in newnulldf:
    if col_name not in dates:
        telecom.loc[pd.isnull(telecom[col_name]), [col_name]] = 0
    else:
        telecom.loc[pd.isnull(telecom[col_name]), [col_name]] = "01/01/1970"

In [103]:
telecom["date_of_last_rech_6"] = pd.to_datetime(telecom["date_of_last_rech_6"], format="%m/%d/%Y")
telecom["date_of_last_rech_7"] = pd.to_datetime(telecom["date_of_last_rech_7"], format="%m/%d/%Y")
telecom["date_of_last_rech_8"] = pd.to_datetime(telecom["date_of_last_rech_8"], format="%m/%d/%Y")

In [104]:
telecom.isnull().sum()

mobile_number       0
onnet_mou_6         0
onnet_mou_7         0
onnet_mou_8         0
offnet_mou_6        0
offnet_mou_7        0
offnet_mou_8        0
roam_ic_mou_6       0
roam_ic_mou_7       0
roam_ic_mou_8       0
roam_og_mou_6       0
roam_og_mou_7       0
roam_og_mou_8       0
loc_og_t2t_mou_6    0
loc_og_t2t_mou_7    0
loc_og_t2t_mou_8    0
loc_og_t2m_mou_6    0
loc_og_t2m_mou_7    0
loc_og_t2m_mou_8    0
loc_og_t2f_mou_6    0
loc_og_t2f_mou_7    0
loc_og_t2f_mou_8    0
loc_og_t2c_mou_6    0
loc_og_t2c_mou_7    0
loc_og_t2c_mou_8    0
loc_og_mou_6        0
loc_og_mou_7        0
loc_og_mou_8        0
std_og_t2t_mou_6    0
std_og_t2t_mou_7    0
                   ..
vol_2g_mb_7         0
vol_2g_mb_8         0
vol_3g_mb_6         0
vol_3g_mb_7         0
vol_3g_mb_8         0
night_pck_user_6    0
night_pck_user_7    0
night_pck_user_8    0
monthly_2g_6        0
monthly_2g_7        0
monthly_2g_8        0
sachet_2g_6         0
sachet_2g_7         0
sachet_2g_8         0
monthly_3g

In [105]:
telecom[telecom == 0].count(axis=0).sort_values(ascending=False)

og_others_7            29815
og_others_8            29799
night_pck_user_8       29700
night_pck_user_7       29683
night_pck_user_6       29668
spl_ic_mou_7           28331
sachet_3g_8            28252
sachet_3g_7            28022
sachet_3g_6            28001
isd_og_mou_8           27881
spl_ic_mou_8           27563
isd_og_mou_7           27476
sep_vbc_3g             27398
churn                  27390
isd_og_mou_6           27341
monthly_2g_8           26909
monthly_2g_6           26651
monthly_2g_7           26526
monthly_3g_8           26496
monthly_3g_6           26401
monthly_3g_7           26330
std_og_t2f_mou_8       25393
std_og_t2f_mou_7       25115
count_rech_3g_8        25086
std_og_t2f_mou_6       25020
count_rech_3g_6        24802
count_rech_3g_7        24772
sachet_2g_6            24754
sachet_2g_7            24525
roam_ic_mou_7          24244
                       ...  
offnet_mou_8            1454
loc_og_t2m_mou_6        1396
loc_og_t2m_mou_7        1268
loc_ic_t2m_mou

###### Finding churned customers based on data in the "Action" month

In [337]:
#Getting rows where churn = 0 
telecom_zeros = telecom.loc[(telecom["churn"] == 0),:]
telecom_9_ones = telecom.loc[(telecom["churn"] == 1),:]                 #to be concat

In [338]:
telecom_zeros.shape

(27390, 128)

In [339]:
#If the data of "Action" months are zero
telecom_zeros["churn"] = (telecom_zeros["total_ic_mou_8"]==0) & (telecom_zeros["total_og_mou_8"]==0) & (telecom_zeros["vol_3g_mb_8"]==0) & (telecom_zeros["vol_2g_mb_8"]==0)

C:\Users\Debayan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [340]:
telecom_8_ones = telecom_zeros.loc[(telecom_zeros["churn"] == 1),:]             #to be concat

In [341]:
telecom_8_ones.shape

(453, 128)

In [342]:
telecom_final =  telecom_zeros.loc[(telecom_zeros["churn"] == 0),:]

In [343]:
telecom_final.shape

(26937, 128)

###### Creating derived attributes from original attributes

In [344]:
telecom_final.columns

Index(['mobile_number', 'onnet_mou_6', 'onnet_mou_7', 'onnet_mou_8',
       'offnet_mou_6', 'offnet_mou_7', 'offnet_mou_8', 'roam_ic_mou_6',
       'roam_ic_mou_7', 'roam_ic_mou_8',
       ...
       'sachet_3g_6', 'sachet_3g_7', 'sachet_3g_8', 'aon', 'aug_vbc_3g',
       'jul_vbc_3g', 'jun_vbc_3g', 'sep_vbc_3g', 'avg_rech_amt_6_7', 'churn'],
      dtype='object', length=128)